In [1]:
import os
from dotenv import load_dotenv

load_dotenv("../.env")
from langchain.chat_models import ChatOpenAI
from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.llms import OpenAI
from langchain.agents.tools import Tool
from langchain import LLMMathChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_extraction_chain, create_extraction_chain_pydantic
from langchain.prompts import ChatPromptTemplate
from dotenv import load_dotenv
import os
from githubanalyzer import GithubProfileAnalyzer
import json
import requests

api_key = os.getenv("OPENAI_API_KEY")
serper_api_key = os.getenv("SERP_API_KEY")

In [2]:
def extract_resume_information(pdf_path):
    # Load PDF
    loader = PyPDFLoader(pdf_path)
    final = loader.load()
    resume = final[0].page_content
    llm = ChatOpenAI(temperature=0.0)

    pydantic_parser = PydanticOutputParser(pydantic_object=Applicant)
    format_instructions = pydantic_parser.get_format_instructions()

    template_string = """ You are a professional HR that and information extractor that can analyze the resume of the person and extract the contents as given in the instructions below
    Resume: ```{resume}```
    {format_instructions}
    """
    prompt = ChatPromptTemplate.from_template(template=template_string)
    messages = prompt.format_messages(resume=resume, format_instructions=format_instructions)

    output = llm(messages)

    return pydantic_parser.parse(output.content)

In [3]:
extract_resume_information('new_pdf\Poojan_vig resume.pdf')

NameError: name 'PyPDFLoader' is not defined

In [ ]:
def analyze_github_profile(profile_url):
    load_dotenv(".env")
    os.environ["LANGCHAIN_WANDB_TRACING"] = "true"
    os.environ["WANDB_PROJECT"] = "Claude"
    anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
    serper_api_key = os.getenv("SERP_API_KEY")

    analyzer = GithubProfileAnalyzer(anthropic_api_key=anthropic_api_key, serper_api_key=serper_api_key)
    return analyzer.analyze(profile_url)


In [ ]:
resume_tool = Tool(
    name="Resume Extraction",
    func=extract_resume_information,
    description="Useful for extracting information from a resume. The function takes a path to a PDF file as input."
)

# Define the tool for analyzing a GitHub profile
github_tool = Tool(
    name="GitHub Profile Analysis",
    func=analyze_github_profile,
    description="Useful for analyzing a GitHub profile. The function takes a GitHub profile URL as input."
)

tools = [resume_tool, github_tool]

In [ ]:
system_message=SystemMessage(
    content="""You are a world class HR, who hires people based on their talent and skills; 
            you do not make things up, you will try as hard as possible to learn facts, data and infromation from their resumes and github
                        
            Please make sure you complete the objective above with the following rules:
            1/ Extract all the information from the resume and understand it
            2/ From the extracted data use the github link to analysis that persons github profile
            3/ After collecting all the information and analysing it, you should conclude the summery of that person

In [ ]:
agent_kwargs = {
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
    "system_message": system_message,
}

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k-0613")
memory = ConversationSummaryBufferMemory(
    memory_key="memory", return_messages=True, llm=llm, max_token_limit=1000)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    agent_kwargs=agent_kwargs,
    memory=memory,
)